In [1]:
import streamlit as st
from langchain.agents import initialize_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from sqlalchemy import create_engine
from langchain_groq import ChatGroq
from langchain.prompts import SemanticSimilarityExampleSelector,PromptTemplate,FewShotPromptTemplate
from few_shots import few_shots
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
from langchain.chains import create_sql_query_chain
import chromadb
from langchain import hub

In [12]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

tempdf=f"./agents/pdfs/clove_dental.pdf"
loader=PyPDFLoader(tempdf)
docs=loader.load()
docs


text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=50)
docs_split=text_splitter.split_documents(documents=docs)
docs_split

[Document(metadata={'source': './agents/pdfs/clove_dental.pdf', 'page': 0}, page_content='1\nJOURNAL OF DENTAL HEAL TH & RESEARCH (VOL. 4, ISSUE 1, JAN - JUN 2023)\n2023 PREDICTIONS - A GLIMPSE INTO THINGS TO COME\nLt Gen Dr Vimal Arora\nEditor-in-Chief\nEDITORIAL\nIndia as a developing country has been grappling with\nthe strategies related to Health due to its exploding\npopulation, varying socio-economic scenario, lack of\nawareness and education on health; and the Government\nhas been struggling to provide healthcare infrastructure\nacross the country due to its vastness.\nPoor Oral Health can have negative impact on\nphysical health, including an increased risk of diseases\nsuch as diabetes, heart disease, and respiratory\ninfections. Additionally, oral health can also have\nsignificant impact on mental health and quality of life.\nOral Health has not been getting the right kind of impetus\nas it deserves, however during past two years, the focus\nis shifting, and healthy oral cav

In [2]:
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
system_message = prompt_template.format(dialect="SQLite", top_k=5)
chromadb.api.client.SharedSystemClient.clear_system_cache()

In [4]:
# Sidebar inputs for MySQL connection and Groq API Key
mysql_host = "localhost"
mysql_user = "genai"
mysql_password = "genai"
mysql_db = "offer_prm_uat"
api_key = "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"

query_limit = 100


In [5]:
llm = ChatGroq(groq_api_key=api_key, model="gemma2-9b-it", streaming=True)

In [6]:
def config_mysql_db(mysql_host, mysql_user, mysql_password, mysql_db):
    """Configure MySQL Database connection."""
    if not (mysql_db and mysql_host and mysql_user and mysql_password):
        st.error("Please provide complete MySQL DB configuration.")
        st.stop()
    return SQLDatabase(create_engine(f"mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_db}"))

In [7]:
db = config_mysql_db(mysql_host, mysql_user, mysql_password, mysql_db)

/var/www/learning/genAIApps/langchain-framework/langenv/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:123: SAWarning: Unknown schema content: '  KEY `\t\t'
  self._metadata.reflect(
/var/www/learning/genAIApps/langchain-framework/langenv/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:123: SAWarning: Unknown schema content: '\tblock_access_till_time` (`block_access_till_time`),'
  self._metadata.reflect(


In [8]:
toolkit = SQLDatabaseToolkit(llm=llm, db=db)

In [9]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
to_vectorize = [" ".join(example.values()) for example in few_shots]
vectorstore = FAISS.from_texts(to_vectorize, embeddings, metadatas=few_shots)

/tmp/ipykernel_4104/776014631.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/var/www/learning/genAIApps/langchain-framework/langenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=few_shots,
    embeddings=embeddings,
    vectorstore_cls=FAISS,
    k=5
)


mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Pay attention to use CURDATE() function to get the current date, if the question involves "today".
    
    Use the following format:
    
    Question: Question here
    SQLQuery: Query to run with no pre-amble
    SQLResult: Result of the SQLQuery
    Answer: Final answer here
    
    No pre-amble.
    """

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

few_shot_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        prefix=mysql_prompt,
        suffix=PROMPT_SUFFIX,
        input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
    )

In [11]:
agent = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,  # Enable parsing error handling
    prompt=system_message,  # Use the FewShotPromptTemplate
)

/tmp/ipykernel_4104/2820498969.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


In [15]:
def returnResp(user_input):
    limited_query = f"{user_input} LIMIT {query_limit}"
    # streamlit_callback = StreamlitCallbackHandler(st.container())
    response = agent.run(limited_query)
    # response = response[:500]
    return response

In [16]:
try:
    returnResp("how many clinics we have")
except Exception as e:
    st.error(f"Error: {e}")



> Entering new AgentExecutor chain...
Thought: I need to query the database to find the number of clinics. 
Action: sql_db_list_tables
Action Input: 
Observation: PAYRECO_PRMCASH_PTCASH_CORRCETION, accessories, accessory_incidence, accessory_transfer, acos, addresses, advancereceiptwithfraction, ageing_reports, ageing_treatmentwise_reports, allocation_histories, amc_misc_data, appointment_expired_reasons, aros, aros_acos, aros_delete_ids, asset_accessory_map, asset_category, asset_incidence, asset_incidence_images, asset_transfer, asset_transfer_accessories, asset_transfer_images, assets_master, authorities, automatic_notification, billing, billing_consumed_aligner_stages_mapping, blocked_contact, blocked_emails_temp, call_info_disposition_relations, call_infos, call_qualites, camp_field_dentist_zone_maps, camp_field_dentists, campaigns, camps, categories, categories_seq, cc_groups, cc_intensities, cc_lists, cc_locations, chart_tracker, check_sum_logs, checkup_config, checkup_data, c

2025-01-04 19:04:29.579 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 19:04:29.580 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
